In [ ]:
# 1 Sentiment value
import pandas as pd
import csv
import numpy as np

# Set data directory
input_dir = "data/annotated/annotated_contents.csv"
output_dir = "data/annotated/finale/valuated_contents.csv"

# Create arousal word list
arousal_list = []
dicdir = 'combined_political_arousal.txt'
with open(dicdir, 'r', encoding = 'utf-8-sig') as f:
    for word in f:
        arousal_list.append(word.strip())

In [ ]:
def political_arousal(text_str):
    # Cut text
    import jieba
    import unicodedata
    import string
    text = unicodedata.normalize('NFKC', text_str)
    seg_list = jieba.cut(text, cut_all = False, HMM = True)
    seg_list = [word.lower() for word in seg_list if word not in string.punctuation # remove punctuation
                and not word.isnumeric() # remove digits
                and word not in ['\ue5e5',' ']] 
    arousal_words = [x for x in seg_list if x in arousal_list]
    return len(arousal_words)/np.log(1 + len(seg_list))

In [ ]:
df = pd.read_csv(input_dir, encoding='utf-8')

# Create new output file
with open(output_dir, "w", newline = '',encoding = 'utf-8') as csvfile:
    w = csv.writer(csvfile, quoting=csv.QUOTE_MINIMAL)
    header = ['created_at','topic_country', 'sentiment', 'arousal_degree', 'title','content']
    w.writerow(header)

i = 1
for index, row in df.iterrows():
    topic_country = row['topic_country']
    sentiment = row['sentiment']
    created_at = row['created_at']
    title = row['title']
    content = row['content']
    arousal_degree = political_arousal(title + content)
    if i % 200 == 0:
        print(i,'out of ', len(df))
    i += 1
    
    # Record output 
    records = [created_at,topic_country, sentiment, arousal_degree, title,content]
    with open(output_dir, "a", newline = '',encoding = 'utf-8') as csvfile:
        w = csv.writer(csvfile, quoting=csv.QUOTE_MINIMAL)
        w.writerow(records)